In [ ]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import brightway2 as bw

# Custom utils defined for inter-acv
from brightway2 import *
from lca_algebraic import *
from lca_algebraic.stats import * 
import lca_algebraic
from sympy import init_printing, simplify

In [ ]:
projects.set_current('LCA_algebraic_manual')
USER_DB = 'MyModelName'
bw2setup()
ei37dir = "/user/jlask/Nextcloud/Ecoinvent/ecoinvent 3.7_cutoff_ecoSpold02/datasets"
if 'ecoinvent 3.7_cutoff' in databases:
    print("Database has already been imported")
else:
    ei37 = SingleOutputEcospold2Importer(ei37dir, 'ecoinvent 3.7_cutoff') # You can give it another name of course
    ei37.apply_strategies()
    ei37.statistics()
ei37.write_database() # This will take some time.

In [ ]:
resetDb(USER_DB)
resetParams(USER_DB)

In [ ]:
#biomass-related parameters
yield_harv = newFloatParam(
    'yield_harv', 
    default=14000, min=9500, max=23500,  distrib=DistributionType.TRIANGLE,
    description="yield, harvested biomass")

carbon_seq = newFloatParam(
    'carbon_seq', 
    default=0, min=0, max=2200,  distrib=DistributionType.FIXED,
    description="carbon_seq")

dmshare_stubbles = newFloatParam(
    'dmshare_stubbles', 
    default=0.04, min=0.02, max=0.06,  distrib=DistributionType.TRIANGLE,
    description="dmshare_stubbles")

dmshare_leaves = newFloatParam(
    'dmshare_leaves', 
    default=0.24, min=0.20, max=0.29,  distrib=DistributionType.TRIANGLE,
    description="dmshare_leaves")

RS = newFloatParam(
    'RS', 
    default=0.8, min=0.7, max=1.5,  distrib=DistributionType.TRIANGLE, #ratio of above-ground dry matter to harvested yield for crop; default acc. to IPCC
    description="dmshare_leaves")

N_stubbles = newFloatParam(
    'N_stubbles', 
    default=0.002, min=0.0015, max=0.0025, distrib=DistributionType.TRIANGLE, #N content of stubbles
    description="N_stubbles")

N_leaves = newFloatParam(
    'N_leaves', 
    default=0.007, min=0.006, max=0.008, distrib=DistributionType.TRIANGLE, #N content of leaves 
    description="N_leaves")

N_bgr = newFloatParam(
    'N_bgr', 
    default=0.010, min=0.006, max=0.014, distrib=DistributionType.TRIANGLE, #N content of below-ground biomass acc. to IPCC default
    description="N_bgr")

mass_rhizomes = newFloatParam(
    'mass_rhizomes', 
    default=0.075, min=0.050, max=0.100, distrib=DistributionType.TRIANGLE, #mass of rhizomes per unit (expert estimate)
    description="mass_rhizomes") 

mass_plugs = newFloatParam(
    'mass_plugs', 
    default=0.011389, min=0.009, max=0.013, distrib=DistributionType.TRIANGLE, #mass of seed plugs per unit (expert estimate)
    description="mass_plugs") 

#diesel consumptions for agricultural operations
diesel_ploughing = newFloatParam(
    'diesel_ploughing', 
    default=16.62, min=12.61, max=20.97,  distrib=DistributionType.TRIANGLE,
    description="diesel_ploughing")

diesel_harrowing = newFloatParam(
    'diesel_harrowing', 
    default=14.78, min=10.75, max=19.76,  distrib=DistributionType.TRIANGLE,
    description="diesel_harrowing")

diesel_rhizome_planting = newFloatParam(
    'diesel_rhizome_planting', 
    default=13.57, min=11.13, max=16.56,  distrib=DistributionType.TRIANGLE,
    description="diesel_rhizome_planting")

diesel_plug_planting = newFloatParam(
    'diesel_plug_planting', 
    default=16.16, min=13.25, max=19.71,  distrib=DistributionType.TRIANGLE,
    description="diesel_plug_planting")

diesel_rolling = newFloatParam(
    'diesel_rolling', 
    default=2.15, min=1.75, max=2.66,  distrib=DistributionType.TRIANGLE,
    description="diesel_rolling")

diesel_appplantprot = newFloatParam(
    'diesel_appplantprot', 
    default=2.11, min=1.5, max=2.8,  distrib=DistributionType.TRIANGLE,
    description="diesel_appplantprot")

diesel_fertilisation = newFloatParam(
    'diesel_fertilisation', 
    default=0.2, min=0.15, max=0.25,  distrib=DistributionType.TRIANGLE,
    description="diesel_fertilisation")

diesel_combine = newFloatParam(
    'diesel_combine', 
    default=9.03, min=7.56, max=10.50,  distrib=DistributionType.TRIANGLE,
    description="diesel_combine")

diesel_swathing = newFloatParam(
    'diesel_swathing', 
    default=2.94, min=2.46, max=3.42,  distrib=DistributionType.TRIANGLE,
    description="diesel_swathing")

diesel_baling = newFloatParam(
    'diesel_baling', 
    default=0.84, min=0.6, max=1.1,  distrib=DistributionType.TRIANGLE,
    description="diesel_baling")

diesel_baleload = newFloatParam(
    'diesel_baleload', 
    default=0.42, min=0.35, max=0.49,  distrib=DistributionType.TRIANGLE,
    description="diesel_baleload")

diesel_recultivation = newFloatParam(
    'diesel_recultivation', 
    default=16.62, min=12.61, max=20.97,  distrib=DistributionType.TRIANGLE,
    description="diesel_recultivation")

#management
cult_per = newFloatParam(
    'cult_per', 
        default=20, min=5, max=25, std=3,  distrib=DistributionType.NORMAL,
    description="cultivation period in years")

num_rhizomes = newFloatParam(
    'num_rhizomes', 
    default=15000, min=14000, max=16000,  distrib=DistributionType.FIXED,
    description="num_rhizomes")

bale_m = newFloatParam(
    'bale_m', 
    default=615, 
    description="bale_m")

#fertilisation and liming
freq_Nfertilisation = newFloatParam(
    'freq_Nfertilisation', 
    default=0.0000001, min=0, max=0.5,  distrib=DistributionType.TRIANGLE,
    description="freq_Nfertilisation") 

freq_Kfertilisation = newFloatParam(
    'freq_Kfertilisation', 
    default=1, min=0.75, max=1,  distrib=DistributionType.TRIANGLE,
    description="freq_Kfertilisation") 

freq_Pfertilisation = newFloatParam(
    'freq_Pfertilisation', 
    default=0.25, min=0.25, max=0.5,  distrib=DistributionType.TRIANGLE,
    description="freq_Pfertilisation") 

N_fert = newFloatParam(
    'N_fert', 
    default=0.00000001, min=0, max=600,  distrib=DistributionType.TRIANGLE,
    description="N_fert") 

K_fert = newFloatParam(
    'K_fert', 
    default=2380, min=1700, max=3500,  distrib=DistributionType.TRIANGLE,
    description="K_fert")

P_fert = newFloatParam(
    'P_fert', 
    default=280, min=200, max=350,  distrib=DistributionType.TRIANGLE,
    description="P_fert")

lime_app = newFloatParam(
    'lime_app', 
    default=1, min=0, max=1,  distrib=DistributionType.FIXED,
    description="lime_app")

lime_fert = newFloatParam(
    'lime_fert', 
    default=2160, min=1500, max=2500,  distrib=DistributionType.TRIANGLE,
    description="lime_fert") 

#application of herbicides
herb_app =  newFloatParam(
    'herb_app', 
    default=3, min=1.5, max=5,  distrib=DistributionType.TRIANGLE,
    description="herb_app") 

amount_glyphosate =  newFloatParam(
    'amount_glyphosate', 
    default=0.324, min=0.2916, max=0.3564,  distrib=DistributionType.TRIANGLE,
    description="amount_glyphosate") 

amount_pendimethalin =  newFloatParam(
    'amount_pendimethalin', 
    default=0.261, min=0.2349, max=0.2871,  distrib=DistributionType.TRIANGLE,
    description="amount_pendimethalin")

amount_phenoxy =  newFloatParam(
    'amount_phenoxy', 
    default=0.33, min=0.297, max=0.363,  distrib=DistributionType.TRIANGLE,
    description="amount_phenoxy")

amount_pesticide_unspec =  newFloatParam(
    'amount_pesticide_unspec', 
    default=0.011, min=0.0099, max=0.0121,  distrib=DistributionType.TRIANGLE,
    description="amount_pesticide_unspec") 

#transport distances
distance_rhizomesplugs = newFloatParam(
    'distance_rhizomesplug', 
    default=800, min=20, max=2000,  distrib=DistributionType.TRIANGLE,
    description="distance_rhizomesplugs")

distance_rhizomesplugs_field = newFloatParam(
    'distance_rhizomesplugs_field', 
    default=2, min=0.5, max=10,  distrib=DistributionType.TRIANGLE,
    description="distance_rhizomesplugs_field")

distance_cust = newFloatParam(
    'distance_cust', 
    default=60, min=20, max=100, std=18,  distrib=DistributionType.NORMAL,
    description="distance_cust")

distance_generic_input = newFloatParam(
    'distance_generic_input', 
    default=150, 
    description="distance_generic_input") 

#emission factors
EF_1 = newFloatParam(  #amount of N2O emitted from nitrogen input - default value acc. to IPCC 2019
    'EF_1', 
    default=0.01,
    description="EF_1")

Frac_GASF = newFloatParam(  #fraction of N fertiliser lost through volatilisation (NH3 and NOx) - default value acc. to IPCC 2019
    'Frac_GASF', 
    default=0.11,
    description="Frac_GASF")

EF_4 = newFloatParam(  #N2O from N volatilised and redeposited - acc. to IPCC 2019
    'EF_4', 
    default=0.010,
    description="EF_4")

Frac_LEACH = newFloatParam(  #fraction of N lost through leaching (NO3) - default value acc. to IPCC 2019
    'Frac_LEACH', 
    default=0.24,
    description="Frac_LEACH")

EF_5 = newFloatParam(  #N2O from N leached - acc. to IPCC 2019
    'EF_5', 
    default=0.011,
    description="EF_5")

EF_limestone = newFloatParam(  #CO2 from liming - acc. to IPCC 2006
    'EF_limestone', 
    default=0.12,
    description="EF_limestone")

#speficied emission factors (for Calcium Ammonium Nitrate - CAN)
EF_1_CAN = newFloatParam(  #amount of N2O emitted from CAN fertiliser - acc. to IPCC 2019
    'EF_1_CAN', 
    default=0.007,
    description="EF_1_CAN")

Frac_GASF_CAN = newFloatParam(  #fraction of CAN fertiliser valatalised as NH3 and NOx
    'Frac_GASF_CAN', 
    default=0.05,
    description="Frac_GASF_CAN")


In [ ]:
list_parameters()

In [ ]:
#background inventory for agricultural operations
diesel_input = findTechAct("market for diesel", loc ="Europe without Switzerland")
tillage_machinery = findTechAct("agricultural machinery production, tillage", loc = "CH")
unspecified_machinery = findTechAct("agricultural machinery production, unspecified", loc = "CH")
shed = findTechAct("shed construction", loc = "CH")
tractor = findTechAct("tractor production, 4-wheel, agricultural", loc = "CH")
harvester = findTechAct("harvester production", loc = "CH")

#background inventory for miscanthus production, excl. agricultural operations
ammonia = Database("biosphere3").get(code = "87883a4e-1e3e-4c9d-90c0-f1bea36f8014")
nitrate = Database("biosphere3").get(code = "b9291c72-4b1d-4275-8068-4c707dc3ce33")
N2O = Database("biosphere3").get(code = "20185046-64bb-4c09-a8e7-e8a9e144ca98") 
CO2 = Database("biosphere3").get(code = "349b29d1-3e58-4c66-98b9-9d1a076efd2e")
CH4 = Database("biosphere3").get(code = "0795345f-c7ae-410c-ad25-1845784c75f5")
lime = Database("ecoinvent 3.7_cutoff").get(code = "2da106ba30d96293e94990c07749751d") 
liming = Database("ecoinvent 3.7_cutoff").get(code = "9081528a0316182e44aee1890f5c406b") 
nitrogen = Database("ecoinvent 3.7_cutoff").get(code = "c7f186a6f67a9d1848988d75db437a9b")
potassium = Database("ecoinvent 3.7_cutoff").get(code = "6e29e111ff610d4e3cf5425fcff3265a")
phosphorus = Database("ecoinvent 3.7_cutoff").get(code = "6f53e20c4ee6c909ff144580e5b604cb")
CAN = Database("ecoinvent 3.7_cutoff").get(code = "51de07214033f9977267fa75e50b109d")  
KCL = Database("ecoinvent 3.7_cutoff").get(code = "dc0391465dec22a2e6bfb6be14f0762e")  
MAP = Database("ecoinvent 3.7_cutoff").get(code = "de38cd052c4146c9fbce57e03a6ea0a8")  
transport = Database("ecoinvent 3.7_cutoff").get(code = "345d308163d966d6c75f4c15d906e7af")  
glyphosate = Database("ecoinvent 3.7_cutoff").get(code = "fc66e7cb2a4c7d97e163727e8af0aa14")  
pendimethalin = Database("ecoinvent 3.7_cutoff").get(code = "85378d56e3f21f2ff9ca2f6b5630abf1")
pesticide_unspec = Database("ecoinvent 3.7_cutoff").get(code = "a5e6591d95ddd81de2d3075f6fe0ee96")
phenoxy_comp = Database("ecoinvent 3.7_cutoff").get(code = "9ac52fbc0be5c750736f0244a1a57276")  
rhizomes = Database("ecoinvent 3.7_cutoff").get(code = "b2409a4e97206de491893fe495fcfd32")  
agri_trans = Database("ecoinvent 3.7_cutoff").get(code = "c8b36e792bb6b8249077e6cfd97bb86e")  
electricity_medvoltage = Database("ecoinvent 3.7_cutoff").get(code = "f0a831380c76f25ae02b1538b418d7d6")  
extrusion_plastic = Database("ecoinvent 3.7_cutoff").get(code = "1dace3ae96abb71bab013d0fc90b3e3a")  
greenhouse = Database("ecoinvent 3.7_cutoff").get(code = "11b46dce22c8d68ace931cf514707dad")  
heat = Database("ecoinvent 3.7_cutoff").get(code = "454fc0db3b02c7ae58e45a17e3b53ca9")  
irrigation = Database("ecoinvent 3.7_cutoff").get(code = "ee912a134574e35c0bbdfab000f35564")  
peat = Database("ecoinvent 3.7_cutoff").get(code = "34441f756d8fed0532e54cc3a7d59eb0")  
pe = Database("ecoinvent 3.7_cutoff").get(code = "db53dd7456969fcb4134728a55fdb7b5")  
pet = Database("ecoinvent 3.7_cutoff").get(code = "409e86951fb6b42b2b89dbf49b61131c")  
pp = Database("ecoinvent 3.7_cutoff").get(code = "982c8a7a849719cd5883b3cbca7cb7df")  
waste_pe = Database("ecoinvent 3.7_cutoff").get(code = "fbd998303eb431ba76d12b6590416d4e") 
waste_pet = Database("ecoinvent 3.7_cutoff").get(code = "3155ad5d7ec9bdc4b6828384beeeafc4")  
waste_pp = Database("ecoinvent 3.7_cutoff").get(code = "287e34218979275c17bb66150c917704")  
hoeing = Database("ecoinvent 3.7_cutoff").get(code = "4d7f5de4b23a98246f9a014500e7bf66")  

In [ ]:
#agricultural operations
diesel = newActivity(USER_DB,
                       "diesel, burned in miscanthus machinery",
                       "MJ",
                       exchanges = {
                           diesel_input: 0.02222,
                           CO2: 0.0693170234455,
                           CH4: 0.00000286952089704, 
                           N2O: 0.00000266564729867                           
                       })


ploughing = newActivity(USER_DB,
                       "ploughing, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_ploughing * 45.8,
                           tillage_machinery: 2.16,
                           shed: 0.00801,
                           tractor: 1.55
                       })

harrowing = newActivity(USER_DB,
                       "harrowing, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_harrowing * 45.8,
                           tillage_machinery: 3.29,
                           shed: 0.0053,
                           tractor: 1.99
                       })

rhizome_planting = newActivity(USER_DB,
                       "rhizome_planting, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_rhizome_planting * 45.8,
                           unspecified_machinery: 1.99,
                           shed: 0.0132,
                           tractor: 2.43
                       })

plug_planting = newActivity(USER_DB,
                       "plug_planting, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_plug_planting * 45.8,
                           unspecified_machinery: 1.99,
                           shed: 0.0132,
                           tractor: 2.43
                       })

rolling = newActivity(USER_DB,
                       "rolling, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_rolling * 45.8,
                           tillage_machinery: 1.82,
                           shed: 0.00534,
                           tractor: 0.412
                       })

applicationPPS = newActivity(USER_DB,
                       "applicationPPS, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_appplantprot * 45.8,
                           unspecified_machinery: 0.53,
                           shed: 0.00198,
                           tractor: 0.321
                       })

fertilisation = newActivity(USER_DB,
                       "fertilisation, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_fertilisation * 45.8,
                           unspecified_machinery: 0.241,
                           shed: 0.00171,
                           tractor: 0.687
                       })

combine = newActivity(USER_DB,
                       "combine harvesting, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_combine * 45.8,
                           harvester: 6.30,
                           shed: 0.00858
                       })

swathing = newActivity(USER_DB,
                       "swathing, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_swathing * 45.8,
                           unspecified_machinery: 0.404,
                           shed: 0.00426,
                           tractor: 0.458
                       })

baling = newActivity(USER_DB,
                       "baling, miscanthus",
                       "unit",
                       exchanges = {
                           diesel: diesel_baling * 45.8,
                           unspecified_machinery: 0.152,
                           shed: 0.00025,
                           tractor: 0.0668
                       })

bale_loading = newActivity(USER_DB,
                       "bale_loading, miscanthus",
                       "unit",
                       exchanges = {
                           diesel: diesel_baleload * 45.8,
                           unspecified_machinery: 0.0077,
                           shed: 0.0000667,
                           tractor: 0.0105
                       })

recultivation = newActivity(USER_DB,
                       "recultivation, miscanthus",
                       "ha",
                       exchanges = {
                           diesel: diesel_recultivation * 45.8,
                           tillage_machinery: 2.16,
                           shed: 0.00801,
                           tractor: 1.55
                       })

#seed plugs
plugs = newActivity(USER_DB,
                       "plug_production, miscanthus",
                       "unit",
                       exchanges = {
                           #seedplug production
                           electricity_medvoltage: 55.00009*3.6/15000,
                           extrusion_plastic: 20.33335/15000,
                           greenhouse: 2.54630/15000,
                           heat: 6600.0112/15000,
                           irrigation: 1000.00170/1000/15000,
                           nitrogen: 0.008/15000,
                           potassium: 0.016/15000,
                           phosphorus: 0.00027/15000,
                           peat: 0.600/15000,
                           pe: 20.8335/3/15000,
                           pet: 20.8335/3/15000, 
                           pp: 20.8335/3/15000,
                           waste_pe: 20.8335/3/15000,
                           waste_pet: 20.8335/3/15000,
                           waste_pp: 20.8335/3/15000,
                           #seed production
                           N2O: (1.98+0.69866)/6000/15000,                       
                           ploughing: 1/6000/15000,
                           harrowing: 1/6000/15000,
                           applicationPPS: 2/6000/15000,
                           fertilisation: 2/6000/15000,
                           hoeing: 1/6000/15000, 
                           irrigation: ((5710+5600)/0.45)/1000/6000/15000,
                           nitrogen: 180/6000/15000,
                           potassium: 360/6000/15000,
                           phosphorus: 90/6000/15000
                           })

In [ ]:
#### inventory of baseline model (generic fertilisers; rhizome-based)
miscanthus_baled_at_cust_gen_rhizome = newActivity( #generic fertilisers, rhizome-based
    USER_DB, 
    "miscanthus_baled_at_cust_gen_rhizome",
    "kg",     
    exchanges = {
        nitrate:(((N_fert #Nitrogen input from synthetic fertiliser
                 +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_leaves)*(cult_per-1.5))*N_leaves) #Nitrogen input from leaves
                 +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_stubbles)*(cult_per-1.5))* N_stubbles)#Nitrogen input from stubbles 
                 +((yield_harv/(1-dmshare_leaves-dmshare_stubbles))* RS * N_bgr)) #Nitrogen input from below-ground biomass
                 * Frac_LEACH * 64/14)/((yield_harv)*(cult_per-1.5))), 
        N2O:(
                   ((
                    (N_fert #Nitrogen input from synthetic fertiliser
                    +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_leaves)*(cult_per-1.5))*N_leaves) #Nitrogen input from leaves
                    +((((yield_harv/(1-dmshare_leaves-dmshare_stubbles))*dmshare_stubbles)*(cult_per-1.5)) * N_stubbles)#Nitrogen input input from stubbles
                    +((yield_harv/(1-dmshare_leaves-dmshare_stubbles))* RS * N_bgr)) #Nitrogen input from below-ground biomass
                    *((EF_1 + Frac_LEACH * EF_5)*44/28) #direct N2O from nitrogen input + indirect N2O from leaching
                    )
                    +((N_fert) * Frac_GASF * EF_4 * 44/28) #indirect N2O from gasification 
                   )   
                /((yield_harv)*(cult_per-1.5)) #divided by total yield    
            ),        
        CO2: ((((lime_fert * EF_limestone * lime_app) + (-carbon_seq * cult_per)) * 44/12)/((yield_harv)*(cult_per-1.5))), #CO2 from liming + SOC
        lime: ((lime_fert * lime_app)/((yield_harv)*(cult_per-1.5))),
        nitrogen: ((N_fert)/((yield_harv)*(cult_per-1.5))),
        potassium: ((K_fert)/((yield_harv)*(cult_per-1.5))),
        phosphorus: ((P_fert)/((yield_harv)*(cult_per-1.5))),
        glyphosate: (herb_app*amount_glyphosate/((yield_harv)*(cult_per-1.5))),
        pendimethalin: (herb_app*amount_pendimethalin/((yield_harv)*(cult_per-1.5))),
        pesticide_unspec: (herb_app*amount_pesticide_unspec/((yield_harv)*(cult_per-1.5))),
        phenoxy_comp: (herb_app*amount_phenoxy/((yield_harv)*(cult_per-1.5))),
        rhizomes: (num_rhizomes/((yield_harv)*(cult_per-1.5))),
        agri_trans: ((num_rhizomes * mass_rhizomes / 1000 * distance_rhizomesplugs_field)/((yield_harv)*(cult_per-1.5))),
        transport: ((num_rhizomes * mass_rhizomes / 1000 * distance_rhizomesplugs) #rhizomes
                    + ((((amount_glyphosate/0.364*2.25)+(amount_pendimethalin/0.45*2.9)+(0.125/0.4*amount_pesticide_unspec)+(amount_phenoxy/0.5*3.3))*herb_app)/1000*distance_generic_input) #pesticides 
                    + ((lime_fert / 54 * 100) / 1000 * distance_generic_input) #lime  
                    + (((yield_harv)*(cult_per-1.5)) / 1000 * distance_cust))/((yield_harv)*(cult_per-1.5)),#biomass transport
        liming: (1 * lime_app)/((yield_harv)*(cult_per-1.5)),
        ploughing: 1/((yield_harv)*(cult_per-1.5)),
        harrowing: 1/((yield_harv)*(cult_per-1.5)),
        rhizome_planting: 1/((yield_harv)*(cult_per-1.5)),
        rolling: 1/((yield_harv)*(cult_per-1.5)), 
        applicationPPS: herb_app/((yield_harv)*(cult_per-1.5)), 
        fertilisation: ((freq_Kfertilisation * cult_per) + (freq_Nfertilisation * cult_per))/((yield_harv)*(cult_per-1.5)), #potassium + nitrogen
        combine: (cult_per-1)/((yield_harv)*(cult_per-1.5)),
        swathing: (cult_per-1)/((yield_harv)*(cult_per-1.5)),
        baling: 1/bale_m,
        bale_loading: 1/bale_m,
        recultivation: 1/((yield_harv)*(cult_per-1.5))
    })


In [ ]:
#### inventory of specific-fertiliser model (rhizome-based; calcium ammonium nitrate, potassium chloride and monoammonium phosphate)
miscanthus_baled_at_cust_spec = newActivity(
    USER_DB, 
    "miscanthus_baled_at_cust_spec",
    "kg",     
    exchanges = {
        nitrate:(((N_fert #Nitrogen input from synthetic fertiliser
                 +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_leaves)*(cult_per-1.5))*N_leaves) #Nitrogen input from leaves
                 +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_stubbles)*(cult_per-1.5))* N_stubbles)#Nitrogen input from stubbles 
                 +((yield_harv/(1-dmshare_leaves-dmshare_stubbles))* RS * N_bgr)) #Nitrogen input from below-ground biomass
                 * Frac_LEACH * 64/14)/((yield_harv)*(cult_per-1.5))), 
        N2O:(
                   ((
                    (N_fert #Nitrogen input from synthetic fertiliser
                    +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_leaves)*(cult_per-1.5))*N_leaves) #Nitrogen input from leaves
                    +((((yield_harv/(1-dmshare_leaves-dmshare_stubbles))*dmshare_stubbles)*(cult_per-1.5)) * N_stubbles)#Nitrogen input from stubbles
                    +((yield_harv/(1-dmshare_leaves-dmshare_stubbles))* RS * N_bgr)) #Nitrogen input from below-ground biomass
                    *((EF_1 + Frac_LEACH * EF_5)*44/28) #direct N2O from nitrogen input + indirect N2O from leaching
                    )
                    +((N_fert) * Frac_GASF_CAN * EF_4 * 44/28) #indirect N2O from gasification 
                   )   
                /((yield_harv)*(cult_per-1.5)) #divided by total yield    
            ),
        
        CO2: (((lime_fert * EF_limestone * lime_app) + (-carbon_seq * cult_per)) * 44/12)/((yield_harv)*(cult_per-1.5)), #CO2 from liming + SOC
        lime: (lime_fert * lime_app)/((yield_harv)*(cult_per-1.5)),
        CAN: ((N_fert)/0.27)/((yield_harv)*(cult_per-1.5)),
        KCL: ((K_fert)/0.60)/((yield_harv)*(cult_per-1.5)),
        MAP: ((P_fert)/0.52)/((yield_harv)*(cult_per-1.5)),
        glyphosate: (herb_app*amount_glyphosate)/((yield_harv)*(cult_per-1.5)),
        pendimethalin: (herb_app*amount_pendimethalin)/((yield_harv)*(cult_per-1.5)),
        pesticide_unspec: (herb_app*amount_pesticide_unspec)/((yield_harv)*(cult_per-1.5)),
        phenoxy_comp: (herb_app*amount_phenoxy)/((yield_harv)*(cult_per-1.5)),
        rhizomes: num_rhizomes/((yield_harv)*(cult_per-1.5)),
        agri_trans: ((num_rhizomes * mass_rhizomes / 1000 * distance_rhizomesplugs_field)/((yield_harv)*(cult_per-1.5))),
        transport: ((num_rhizomes * mass_rhizomes / 1000 * distance_rhizomesplugs) #rhizomes
                    + ((((amount_glyphosate/0.364*2.25)+(amount_pendimethalin/0.45*2.9)+(0.125/0.4*amount_pesticide_unspec)+(amount_phenoxy/0.5*3.3))*herb_app)/1000*distance_generic_input) #pesticides 
                    + ((N_fert/27*100)/1000*distance_generic_input) #Nfertilizer     
                    + ((K_fert/60*100)/1000*distance_generic_input) #Kfertilizer     
                    + ((P_fert/52*100)/1000*distance_generic_input) #Pfertilizer
                    + ((lime_fert / 54 * 100) / 1000 * distance_generic_input) #lime  
                    + ((yield_harv*(cult_per-1.5)) / 1000 * distance_cust))/((yield_harv)*(cult_per-1.5)),#biomass transport
        liming: (1 * lime_app)/((yield_harv)*(cult_per-1.5)),
        ploughing: 1/((yield_harv)*(cult_per-1.5)),
        harrowing: 1/((yield_harv)*(cult_per-1.5)),
        rhizome_planting: 1/((yield_harv)*(cult_per-1.5)),
        rolling: 1/((yield_harv)*(cult_per-1.5)), 
        applicationPPS: herb_app/((yield_harv)*(cult_per-1.5)), 
        fertilisation: ((freq_Kfertilisation * cult_per) + (freq_Nfertilisation * cult_per))/((yield_harv)*(cult_per-1.5)), #potassium + nitrogen
        combine: (cult_per-1)/((yield_harv)*(cult_per-1.5)),
        swathing: (cult_per-1)/((yield_harv)*(cult_per-1.5)),
        baling: 1/bale_m,
        bale_loading: 1/bale_m,
        recultivation: 1/((yield_harv)*(cult_per-1.5))         
    })


In [ ]:
#### inventory of seed-plug-based model (generic fertilisers)
miscanthus_baled_at_cust_gen_plug = newActivity(
    USER_DB, 
    "miscanthus_baled_at_cust_gen_plug",
    "kg",     
    exchanges = {
        nitrate:(((N_fert #Nitrogen input from synthetic fertiliser
                 +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_leaves)*(cult_per-1.5))*N_leaves) #Nitrogen input from leaves
                 +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_stubbles)*(cult_per-1.5))* N_stubbles)#Nitrogen input from stubbles 
                 +((yield_harv/(1-dmshare_leaves-dmshare_stubbles))* RS * N_bgr)) #Nitrogen input from below-ground biomass
                 * Frac_LEACH * 64/14)/((yield_harv)*(cult_per-1.5))), 
        N2O:(
                   ((
                    (N_fert #Ninput - Nitrogen fertiliser
                    +(((yield_harv/(1-dmshare_leaves-dmshare_stubbles)*dmshare_leaves)*(cult_per-1.5))*N_leaves) #N input from leaves
                    +((((yield_harv/(1-dmshare_leaves-dmshare_stubbles))*dmshare_stubbles)*(cult_per-1.5)) * N_stubbles)#N input from stubbles
                    +((yield_harv/(1-dmshare_leaves-dmshare_stubbles))* RS * N_bgr)) #N input from below-ground biomass
                    *((EF_1 + Frac_LEACH * EF_5)*44/28) #direct N2O from N input + indirect N2O from leaching
                    )
                    +((N_fert) * Frac_GASF * EF_4 * 44/28) #indirect N2O from gasification 
                   )   
                /((yield_harv)*(cult_per-1.5)) #divided by total yield    
            ),
        CO2: ((((lime_fert * EF_limestone * lime_app) + (-carbon_seq * cult_per)) * 44/12)/((yield_harv)*(cult_per-1.5))), #CO2 from liming + SOC
        lime: ((lime_fert * lime_app)/((yield_harv)*(cult_per-1.5))),
        nitrogen: ((N_fert)/((yield_harv)*(cult_per-1.5))),
        potassium: ((K_fert)/((yield_harv)*(cult_per-1.5))),
        phosphorus: ((P_fert)/((yield_harv)*(cult_per-1.5))),
        glyphosate: ((herb_app*amount_glyphosate)/((yield_harv)*(cult_per-1.5))),
        pendimethalin: ((herb_app*amount_pendimethalin)/((yield_harv)*(cult_per-1.5))),
        pesticide_unspec: ((herb_app*amount_pesticide_unspec)/((yield_harv)*(cult_per-1.5))),
        phenoxy_comp: ((herb_app*amount_phenoxy)/((yield_harv)*(cult_per-1.5))),
        plugs: ((num_rhizomes)/((yield_harv)*(cult_per-1.5))),
        agri_trans: ((num_rhizomes * mass_plugs / 1000 * distance_rhizomesplugs_field)/((yield_harv)*(cult_per-1.5))),
        transport: (((num_rhizomes * mass_plugs / 1000 * distance_rhizomesplugs) #rhizomes
                    + ((((amount_glyphosate/0.364*2.25)+(amount_pendimethalin/0.45*2.9)+(0.125/0.4*amount_pesticide_unspec)+(amount_phenoxy/0.5*3.3))*herb_app)/1000*distance_generic_input) #pesticides
                    + ((lime_fert / 54 * 100) / 1000 * distance_generic_input) #lime  
                    + (((yield_harv)*(cult_per-1.5)) / 1000 * distance_cust))/((yield_harv)*(cult_per-1.5))),#biomass transport
        liming: ((1 * lime_app)/((yield_harv)*(cult_per-1.5))),
        ploughing: 1/((yield_harv)*(cult_per-1.5)),
        harrowing: 1/((yield_harv)*(cult_per-1.5)),
        plug_planting: 1/((yield_harv)*(cult_per-1.5)),
        rolling: 1/((yield_harv)*(cult_per-1.5)), 
        applicationPPS: ((herb_app)/((yield_harv)*(cult_per-1.5))), 
        fertilisation: (((freq_Kfertilisation * cult_per) + (freq_Nfertilisation * cult_per))/((yield_harv)*(cult_per-1.5))), #potassium + nitrogen
        combine: ((cult_per-1)/((yield_harv)*(cult_per-1.5))),
        swathing: ((cult_per-1)/((yield_harv)*(cult_per-1.5))),
        baling: 1/bale_m,
        bale_loading: 1/bale_m,
        recultivation: 1/((yield_harv)*(cult_per-1.5))         
    })

In [ ]:
#### printing inventory of considered systems (Table S1)
printAct(miscanthus_baled_at_cust_gen_rhizome, miscanthus_baled_at_cust_gen_plug, miscanthus_baled_at_cust_spec, cult_per = 20)

In [ ]:
#### selecting impact method collection IPCC 2013 no LT - GWP 100a 
impacts = [m for m in bw.methods if 'IPCC 2013 no LT' in str(m)and 'GWP 100a' in str(m)]

In [ ]:
impacts[0]

In [ ]:
##### drawing probabilistic distribution (Figure 2)
graphs(
    miscanthus_baled_at_cust_gen_rhizome, impacts,
    
    # Optionnal layout parameters
    height=7, width=15,nb_cols=2,
    percentiles=[5, 95])

In [ ]:
##### calculating sobol indices; used for Figure 3 (s1)
incer_stochastic_matrix(miscanthus_baled_at_cust_gen_rhizome, impacts, n=10000)

In [ ]:
#### creating simplified model (explaining at least 0.95 of the variation
simplified = sobol_simplify_model(
    miscanthus_baled_at_cust_gen_rhizome,
    impacts,
    n=10000,
    fixed_mode = FixedParamMode.MEDIAN, # replacing minor parameters by median
    min_ratio=0.95,
    num_digits=3)

In [ ]:
# printing simplified model - further simplified for presentation in manuscript (Equation 1)
simplified[0].expr

In [ ]:
#### testing effect of increasing cultivation period length (all other parameters set to default) (Figure S 2)
multiLCAAlgebric(
    [miscanthus_baled_at_cust_gen_rhizome], # The model 
    impacts,
    
    # Parameters of the model
    cult_per=list(range(5,26))
)

In [ ]:
#### comparison between generic-fertiliser-based and specified-fertiliser model (Figure S 3)
compare_simplified(miscanthus_baled_at_cust_spec, impacts, simplified)

In [ ]:
#### comparison between rhizome-based (simplified model) and seed-plug-based cultivation (Figure S 4)
compare_simplified(miscanthus_baled_at_cust_gen_plug, impacts, simplified)